In [10]:
import requests
from bs4 import BeautifulSoup
from requests.exceptions import RequestException

def get_macro_url(food_name, page=0, verbose=False):
    if verbose:
        print(f"*** Searching for {food_name} on page {page}")
    search_base_url = "https://www.fatsecret.es/calorías-nutrición/search?q="
    page_url = "&pg=" + str(page)
    url = search_base_url + food_name + page_url
    response = requests.get(url).text
    soup = BeautifulSoup(response, "html.parser")
    food_links = soup.find_all("td", {"class": "borderBottom"})
    results = []
    for link in food_links:
        a = link.find("a", {"class": "prominent"})
        name = a.text
        href = "https://fatsecret.es" + a["href"]
        results.append((name, href))
    return tuple(results)


def check_100g(food_tuple):
    url = food_tuple[1]
    response = requests.get(url).text
    soup = BeautifulSoup(response, "html.parser")
    serving_size = soup.find("div", {"class": "serving_size black serving_size_value"})
    if serving_size is None:
        return False
    if serving_size.text == "100 g":
        return True
    else:
        return False


def get_macros(food_tuple, verbose=False):
    if verbose:
        print(f"Gathering macros for {food_tuple[0]} -- {food_tuple[1]}")
    url = food_tuple[1]
    try:
        response = requests.get(url, timeout=10)
    except RequestException as e:
        print(f"Error requesting {url}: {e}")
        return None
    if response.status_code != 200:
        print(f"Error requesting {url}: status code {response.status_code}")
        return None
    soup = BeautifulSoup(response.text, "html.parser")
    macros = soup.find("div", {"class": "factPanel"})
    if macros is None:
        print(f"No macros found for {food_tuple[0]}")
        return None
    macro_list = []
    for macro in macros.find_all("div", {"class": "factValue"}):
        macro_list.append(macro.text)
    if len(macro_list) < 4:
        print(f"Not enough macros found for {food_tuple[0]}")
        return None
    cals = float(macro_list[0])
    fat = float(macro_list[1].replace("g", "").replace(",", "."))
    carb = float(macro_list[2].replace("g", "").replace(",", "."))
    prot = float(macro_list[3].replace("g", "").replace(",", "."))

    return {"cals": cals, "fat": fat, "carb": carb, "prot": prot}

# TODO: metadata, brand, etc

*** Searching for chocolate on page 0
Gathering macros for Chocolate -- https://fatsecret.es/calor%C3%ADas-nutrici%C3%B3n/gen%C3%A9rico/chocolate
{'cals': 535.0, 'fat': 29.66, 'carb': 59.4, 'prot': 7.65}

Gathering macros for Chocolate con Leche -- https://fatsecret.es/calor%C3%ADas-nutrici%C3%B3n/milka/chocolate-con-leche/1-porci%C3%B3n
{'cals': 539.0, 'fat': 31.0, 'carb': 57.0, 'prot': 6.5}

Gathering macros for Chocolate con Leche y Almendras  -- https://fatsecret.es/calor%C3%ADas-nutrici%C3%B3n/valor/chocolate-con-leche-y-almendras/100g
{'cals': 567.0, 'fat': 39.0, 'carb': 41.0, 'prot': 11.0}

Gathering macros for Chocolate Negro 99% -- https://fatsecret.es/calor%C3%ADas-nutrici%C3%B3n/lindt/chocolate-negro-99/100g
{'cals': 573.0, 'fat': 49.0, 'carb': 10.0, 'prot': 14.0}

*** Searching for chocolate on page 1
Gathering macros for Chocolate Negro 70% sin Azucar -- https://fatsecret.es/calor%C3%ADas-nutrici%C3%B3n/valor/chocolate-negro-70-sin-azucar/100g
{'cals': 481.0, 'fat': 36.0, 

In [ ]:
import sqlite3
import pandas as pd

def 